In [1]:
import json

In [2]:
import pandas as pd

In [3]:
def get_column_names(schemas, ds_name, sorting_key='column_position'):
    column_details = schemas[ds_name]
    columns = sorted(column_details, key=lambda col: col[sorting_key])
    return [col['column_name'] for col in columns]

In [4]:
file_path = '/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/schemas.json'
schemas = json.load(open(file_path))

In [5]:
schemas.keys()

dict_keys(['departments', 'categories', 'orders', 'products', 'customers', 'order_items'])

In [6]:
orders_columns = get_column_names(schemas, 'orders')
orders_columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [7]:
orders = pd.read_csv('/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/orders/part-00000', names=orders_columns)
orders.head()

In [9]:
customers_columns = get_column_names(schemas, 'customers')
customers_columns

['customer_id',
 'customer_fname',
 'customer_lname',
 'customer_email',
 'customer_password',
 'customer_street',
 'customer_city',
 'customer_state',
 'customer_zipcode']

In [10]:
customers = pd.read_csv('/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/customers/part-00000', names=customers_columns)
customers.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,725


In [11]:
customers = customers.set_index('customer_id')
customers.head()

,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
customer_id,,,,,,,,
1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,725
4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,725


In [12]:
orders = orders.set_index('order_customer_id')
orders.head()

,order_id,order_date,order_status
order_customer_id,,,
11599,1,2013-07-25 00:00:00.0,CLOSED
256,2,2013-07-25 00:00:00.0,PENDING_PAYMENT
12111,3,2013-07-25 00:00:00.0,COMPLETE
8827,4,2013-07-25 00:00:00.0,CLOSED
11318,5,2013-07-25 00:00:00.0,COMPLETE


In [13]:
customer_orders = customers.join(orders, how='inner')
customer_orders.head()

,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode,order_id,order_date,order_status
1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521,22945,2013-12-13 00:00:00.0,COMPLETE
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,15192,2013-10-29 00:00:00.0,PENDING_PAYMENT
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,33865,2014-02-18 00:00:00.0,COMPLETE
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,57963,2013-08-02 00:00:00.0,ON_HOLD
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126,67863,2013-11-30 00:00:00.0,COMPLETE


In [14]:
customer_orders.shape

(68883, 11)

In [15]:
customer_orders = customer_orders.reset_index().rename(columns={'index': 'customer_id'})

In [16]:
customer_orders.\
    groupby('customer_id')['customer_id'].\
    agg(order_count='count')

,order_count
customer_id,
1,1
2,4
3,7
4,6
5,4
...,...
12431,16
12432,10
12433,4


In [17]:
customer_orders1 = customer_orders.\
    groupby('customer_id')['customer_id'].\
    agg(order_count='count').\
    reset_index().\
    query('order_count >= 10')

In [18]:
customer_orders1

,customer_id,order_count
70,71,10
171,172,10
173,174,12
196,197,11
219,221,15
...,...,...
12311,12341,10
12317,12347,10
12375,12406,10
12400,12431,16


In [19]:
customer_orders2 = customer_orders1.sort_values(['order_count'], ascending=False)
customer_orders2.head()

,customer_id,order_count
5885,5897,16
6303,6316,16
12400,12431,16
565,569,16
4310,4320,15


In [20]:
import os

In [21]:
os.makedirs('/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/order_json', exist_ok=True)

In [24]:
orders.reset_index().to_json('/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/order_json/part-00000')


In [27]:
orders.reset_index().to_json('/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/order_json/part-00000', 
    orient='records',
    lines=True
)

In [30]:
pd.read_json('/Users/macintosh/Desktop/data_engineering_Esentials_using/data/retail_db/order_json/part-00000', 
    lines=True
)

,order_customer_id,order_id,order_date,order_status
0,11599,1,2013-07-25 00:00:00.0,CLOSED
1,256,2,2013-07-25 00:00:00.0,PENDING_PAYMENT
2,12111,3,2013-07-25 00:00:00.0,COMPLETE
3,8827,4,2013-07-25 00:00:00.0,CLOSED
4,11318,5,2013-07-25 00:00:00.0,COMPLETE
...,...,...,...,...
68878,778,68879,2014-07-09 00:00:00.0,COMPLETE
68879,1117,68880,2014-07-13 00:00:00.0,COMPLETE
68880,2518,68881,2014-07-19 00:00:00.0,PENDING_PAYMENT
68881,10000,68882,2014-07-22 00:00:00.0,ON_HOLD
